# EVALUATION OF THE CHOOSEN MODELS

In [17]:
# root_path       - path to the wound dataset
# csv_path_folder - path to the folder containing csvs with lists of image names used in given dataset (test, train, validate)
# unet_relu_path  - path to the defined model in .py file. Must be set if given model is being used
# unet_sigm_path  - path to the defined model in .py file. Must be set if given model is being used
# weights_xxx     - path to the file with pretrained weights of given model xxx (DeepLabV3, Unet-Sigmoid, Unet-ReLU)

root_path =         '../input/350pics/dataset'
csv_path_folder =   '../input/wound-dataset-splitedlist/sort whole/'

unet_relu_path =    '../input/sigmoid-model/unet_model.py'
unet_sigm_path =    '../input/nn-model/unet_model.py'

weights_DeepLabV3 = '../input/model-weights/model_weights/deelpabv3_weights.pth'
weights_Unet_ReLU = '../input/model-weights/model_weights_new/unetRelu_weights.pth'
weights_Unet_Sigm = '../input/model-weights/model_weights_new/unetSigmoid_weights.pth'

In [18]:
import os
import csv
import time
import torch
import random
import numpy as np
import pandas as pd
import seaborn as sb
import torch.nn as nn
from skimage import io
import torch.utils.data as data
import matplotlib.pyplot as plt
from torchvision import transforms#, datasets
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, accuracy_score, classification_report, matthews_corrcoef

import warnings
warnings.filterwarnings("ignore")

In [19]:
# Calling these functions fill download and set up given models. If a path to model weights is included, weights will be automaticaly loaded

def createUnet(weights_path=None, model_type='ReLU'):
    assert model_type in ('ReLU', 'Sigmoid'), "Unknown Unet model type"
    # Loading model from the python file
    from shutil import copyfile
    unetpath = unet_relu_path if model_type == 'Unet-Sigmoid' else unet_sigm_path
    copyfile(src = unetpath, dst = "../working/unet_model.py")
    from unet_model import Unet
    model = Unet()
    
    if weights_path is not None:
        model.load_state_dict(torch.load(weights_path, map_location=device))
        
    model = model.to(device)
    return model
    
    
def createDeepLabV3(weights_path=None):
    from torchvision.models.segmentation.deeplabv3 import DeepLabHead
    from torchvision import models
    model = models.segmentation.deeplabv3_resnet101(pretrained=True, progress=True)
    outputchannels=4
    model.classifier = DeepLabHead(2048, outputchannels)
    if weights_path is not None:
        model.load_state_dict(torch.load(weights_path, map_location=device))
    
    model = model.to(device)
    return model

In [20]:
# Creating the class to work with the dataset
class WoundDataset(data.Dataset):

    def __init__(self, root, transform=None, csv_file=None):
        self.img_orig = root + '/imgs'  # folder with the resized images (512 * 512)
        self.img_mask = root + '/masks' # folder with the masks to these images
        self.transform = transform
        
        if csv_file is None:
            self.imglist = os.listdir(self.img_orig)
        else:
            with open(csv_file, newline='') as csvfile:
                spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
                csv_img_list = [row[1] for row in spamreader][1:]
                self.imglist = [filename for filename in os.listdir(self.img_orig) if filename in csv_img_list]
                
    def __len__(self):
        return len(self.imglist)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.img_orig, self.imglist[idx])
        msk_name = os.path.join(self.img_mask, self.imglist[idx])
        
        img = io.imread(img_name)
        msk = io.imread(msk_name)[:,:,:3] # reading just RGB channels (without hue)
        
        if self.transform:
            img = self.transform(img)
            msk = self.transform(msk)

        return img, msk

In [21]:
# Function for coding the input to the NN

def convert_into_1d(old_tensor):
    # pixels that will be marked as a backgroung
    max_pixel_value = (torch.max(torch.flatten(old_tensor))).cpu().numpy()
    color_threshold = 0.5 * max_pixel_value
    
    # 1 - additional channel for the class labeling
    new_tensor = torch.full((old_tensor.shape[0], 1, old_tensor.shape[2], old_tensor.shape[3]), color_threshold)
    
    old_tensor = old_tensor.to(device)
    new_tensor = new_tensor.to(device)
    
    # concatenate two tensors
    old_tensor = torch.cat((old_tensor, new_tensor), dim=1)
    
    # finding the layer with the maximum pixel value
    max_idxs = torch.argmax(old_tensor, dim=1)
                    
    return max_idxs


def converting_to_rgb_layers(x):
    switcher = {
        0: [255, 0, 0], # granulation tissue
        1: [0, 255, 0], # slough tissue
        2: [0, 0, 255], # necrotic tissue
        3: [0, 0, 0]    # background
    }
    return switcher.get(int(x[0]), "error")


# Function for decoding final mask (NN output)
def convert_into_3d(output_mask):
    # create a new 'empty' tensor
    zero_tensor = torch.zeros(output_mask.shape[1], output_mask.shape[2], 1)
    
    # detecting which layer has to be displayed in the picture
    dominant_layers = torch.argmax(torch.tensor(output_mask), dim=0).unsqueeze(2)
    
    # creating a new 3-dimentional tensor
    # first dim - dominant layer, other - empty layers
    three_dim_tensor = torch.cat((dominant_layers, zero_tensor, zero_tensor), dim=2)
    # converting it to numpy array
    numpy_array = three_dim_tensor.detach().numpy()
        
    # changing dominant layers to rgb image
    numpy_array = np.apply_along_axis(converting_to_rgb_layers, -1, numpy_array)
        
    return numpy_array

In [22]:
# Function that helps to display original image, binary truly mask to it, NN generated output mask and the difference between the last two

def display_binary_masks(index, pictures):
    fig, axs = plt.subplots(1, 6, figsize=(24,4))
    for i, picture in enumerate(pictures):
        axs[i].imshow(np.transpose(picture.cpu().numpy(), (1, 2, 0)), cmap = 'gray' if i in (2, 4, 5) else None)
    fig.suptitle('Processing image number ' + str(index + 1), fontweight ='bold')    
    plt.show()   

In [23]:
# Function that helps to convert all classes (r, g, b) to white color 
# Function that helps to create a binary mask

def binarise_tensor(tensor):
    max_color_layer, max_ids = torch.max(tensor, 0)
    return max_color_layer.unsqueeze(0)

In [48]:
# Function that helps to display the confusion matrix

def display_confusion_matrix(truth, prediction, save_name=None):
    cm = confusion_matrix(truth, prediction, labels = [0, 1, 2, 3], normalize = 'true')
    cm_df = pd.DataFrame(cm, 
                         index = ['GRANULATION TISSUE','SLOUGH TISSUE','NECROTIC TISSUE', 'BACKGROUND'],
                         columns = ['GRANULATION TISSUE','SLOUGH TISSUE','NECROTIC TISSUE', 'BACKGROUND'])
    fig = plt.figure(figsize=(7.5,7.5 ))
    sb.heatmap(cm_df, annot=True, fmt='.2%')
    plt.title('Confusion Matrix')
    plt.ylabel('Actal Values')
    plt.xlabel('Predicted Values')
    
    if save_name is not None:
        plt.savefig(save_name)
    plt.show()

In [25]:
# Function that helps to calculate accuracy of the selected model

def calculate_accuracy(index, image, mask, output_mask, display=False):
    binary_mask = binarise_tensor(mask)
    binary_output_mask = binarise_tensor(output_mask)
    difference = torch.logical_xor(binary_mask, binary_output_mask)

    difference = torch.where(difference == True, 1, 0)
    num_diff = torch.sum(difference)
    current_accuracy = 1 - num_diff / (mask.shape[1] * mask.shape[2])
    
    if display:
        display_binary_masks(index, (image, mask, binary_mask, output_mask, binary_output_mask, difference))
        
    return current_accuracy.detach().cpu().numpy()


In [26]:
# Function evaluating score such as Dice score, Iterset-over-Union (IoU) and Matthews correlation coefficient (MCC)
def evaluate_mcc_iou_dice(gt, pred):        
    intersection = np.sum(np.where(np.logical_and(pred==gt, gt<3), 1, 0))
    union = np.sum(np.where(np.logical_or(pred<3, gt<3), 1, 0)) # 3 is the background
    area_pred = np.sum(np.where(pred<3, 1, 0))
    area_gt   = np.sum(np.where(gt  <3, 1, 0))
    ##############3
    
    mcc = matthews_corrcoef(gt, pred)
    iou = intersection / union
    dice= 2*intersection / (area_pred + area_gt)
    
    return [mcc, iou, dice, intersection, union, area_pred, area_gt]

In [27]:
# Main function for model performance testing

def evaluate_and_test_model(model, data_test, display=False):
    accuracy = []
    times = []
    truth = []
    prediction = []
    
    eval_list = []
    true_mask_arr = np.array([])
    pred_mask_arr = np.array([])
    
    model.eval()
    
    for index in range(len(data_test)):
        image, mask = data_test[index]
        
        image = image.to(device)
        start_time = time.time()
        output_mask = model(image.unsqueeze(0))
        if isinstance(output_mask, dict):
            output_mask = output_mask['out']
        output_mask = output_mask.squeeze(0).cpu()
        elapsed_time = time.time() - start_time
        output_mask = torch.from_numpy(np.transpose(convert_into_3d(output_mask), (2,0,1)))
        
        current_accuracy = calculate_accuracy(index, image, mask, output_mask, display)  
#         evaluate_AccPreRecSens(index, image, mask, output_mask) 
        true_mask = torch.flatten(convert_into_1d(mask.unsqueeze(0))).cpu().numpy()
        pred_mask = torch.flatten(convert_into_1d(output_mask.unsqueeze(0))).cpu().numpy()

        accuracy.append(current_accuracy)
        times.append(elapsed_time)
        truth = [*truth, *true_mask]
        prediction = [*prediction, *pred_mask] 
        
        eval_list.append(evaluate_mcc_iou_dice(true_mask, pred_mask))
        # mcc, iou, dice, intersection, union, area_pred, area_gt
        
    eval_list = np.array(eval_list)

    return accuracy, times, truth, prediction, eval_list

In [28]:
# Funciton displaying majority of results
def evaluate(eval_list, truth, prediction):
    print("mcc  avg: {:.3f}".format(np.mean(eval_list[:,0])))
    print("iou  avg: {:.3f}".format(np.mean(eval_list[:,1])))
    print("dice avg: {:.3f}".format(np.mean(eval_list[:,2])))
    
    print()
    print("mcc  median: {:.3f}".format(np.median(eval_list[:,0])))
    print("iou  median: {:.3f}".format(np.median(eval_list[:,1])))
    print("dice median: {:.3f}".format(np.median(eval_list[:,2])))
    
    print()
    print("mcc  whole: {:.3f}".format(matthews_corrcoef(truth, prediction)))
    print("iou  whole: {:.3f}".format(np.sum(eval_list[:,3]) / np.sum(eval_list[:,4  ])))
    print("dice whole: {:.3f}".format(2*np.sum(eval_list[:,3]) / np.sum(eval_list[:,4:6])))
    
    
    print()
    print("classification_report:\n",
          classification_report(
              truth, prediction, digits=3, labels=[0,1,2,3],
              target_names=['granulation tissue',
                            'slough tissue',
                            'necrotic tissue',
                            'background'
                           ], ))

# Evaluation

In [29]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Selected device: {device}')

In [30]:
data_test =  WoundDataset(root = root_path, transform = transforms.ToTensor(), csv_file=csv_path_folder+'test.csv')

# DeepLabV3

In [31]:
model = createDeepLabV3(weights_path= weights_DeepLabV3)
accuracy_DL, times_DL, truth_DL, prediction_DL, eval_list_DL = evaluate_and_test_model(model, data_test, display=False)

In [49]:
evaluate(eval_list_DL, truth_DL, prediction_DL)

print('AREA EXACTNESS: {:.2f}%'.format(np.mean(accuracy_DL) * 100))
print('AVERAGE NECESSARY TIME FOR GENERATING MODEL OUTPUT: {:.3f} second(s)'.format(np.mean(times_DL)))

display_confusion_matrix(truth_DL, prediction_DL, save_name='Confusion_Mattrix_DeepLabv3.png')

# Unet - Sigmoid

In [34]:
model = createUnet(weights_path= weights_Unet_Sigm, model_type='Sigmoid')
accuracy_US, times_US, truth_US, prediction_US, eval_list_US = evaluate_and_test_model(model, data_test, display=False)

In [50]:
evaluate(eval_list_US, truth_US, prediction_US)

print('AREA EXACTNESS: {:.2f}%'.format(np.mean(accuracy_US) * 100))
print('AVERAGE NECESSARY TIME FOR GENERATING MODEL OUTPUT: {:.3f} second(s)'.format(np.mean(times_US)))

display_confusion_matrix(truth_US, prediction_US, save_name='Confusion_Mattrix_Unet-Sigmoid.png')

# Unet - ReLU

In [36]:
model = createUnet(weights_path= weights_Unet_ReLU, model_type='ReLU')
accuracy_UR, times_UR, truth_UR, prediction_UR, eval_list_UR = evaluate_and_test_model(model, data_test, display=False)

In [51]:
evaluate(eval_list_UR, truth_UR, prediction_UR)

print('AREA EXACTNESS: {:.2f}%'.format(np.mean(accuracy_UR) * 100))
print('AVERAGE NECESSARY TIME FOR GENERATING MODEL OUTPUT: {:.3f} second(s)'.format(np.mean(times_UR)))

display_confusion_matrix(truth_UR, prediction_UR, save_name='Confusion_Mattrix_Unet-ReLU.png')

# Image qualitative analysis/comparison

In [54]:
# Marking each class with the color (for example: 0 (granulation) is red color (255, 0, 0) in RGB model)

switcher = {
        0: [255, 0, 0], # granulation tissue
        1: [0, 255, 0], # slough tissue
        2: [0, 0, 255], # necrotic tissue
        3: [0, 0, 0]    # background
    }

# Additional function that helps to match an exact class (0-3) with the color (that it is marked in RGB color model)

def converting_to_rgb_layers(x):
    global switcher
    return switcher.get(int(x[0]), "error")


def convert_into_3d(output_mask):
    # create a new 'empty' tensor
    zero_tensor = torch.zeros(output_mask.shape[1], output_mask.shape[2], 1)
    
    # detecting which layer has to be displayed in the picture
    dominant_layers = torch.argmax(torch.tensor(output_mask), dim=0).unsqueeze(2)
    
    # creating a new 3-dimentional tensor
    # first dim - dominant layer, other - empty layers
    three_dim_tensor = torch.cat((dominant_layers, zero_tensor, zero_tensor), dim=2)
    # converting it to numpy array
    numpy_array = three_dim_tensor.detach().numpy()
        
    # changing dominant layers to rgb image
    numpy_array = np.apply_along_axis(converting_to_rgb_layers, -1, numpy_array)
        
    return numpy_array


# Creating a function for displaying images with their masks
# First 3 images are permanent, the last 3 are changing

def display_data(dataset, model=None, n_base=3, n_rand=4):
    n = n_base + n_rand
    fig, axs = plt.subplots(5,n, figsize=(25,17))
    
    index_list = []
    
    for i in range(5):
        if(i >= n_base and False):
            index=random.randint(n, len(dataset)-1)
            print(index)
        else:
            index = i
    #     index= [5,10,14,20,28, 33][i]
    
        index_list.append(index)

        image, mask = dataset[index]
        original_image = image.to(device).unsqueeze(0)
        original_mask = mask
        
        image = np.transpose(np.array(image), (1,2,0))
        axs[0][i].imshow(image)
        if i == 2:
            axs[0][i].set_title('Input images')

        original_mask = np.transpose(np.array(original_mask), (1,2,0))
        axs[1][i].imshow(original_mask)
        if i == 2:
            axs[1][i].set_title('Original masks')
            
     

    for a in range(3):
        if a==0:
            model = createDeepLabV3(weights_path= weights_DeepLabV3)
            title = 'DeepLabV3'
        elif a==1:
            model = createUnet(weights_path= weights_Unet_Sigm, model_type='Sigmoid')
            title = 'Unet - Sigmoid'
        else:
            model = createUnet(weights_path= weights_Unet_ReLU, model_type='ReLU')
            title = 'Unet - ReLU'
        
        axs[2+a][2].set_title(title)

        for i, index in enumerate(index_list):
            image, mask = dataset[index]
            original_image = image.to(device).unsqueeze(0)
            original_mask = mask
            model.eval()

            with torch.no_grad():            
                output_mask = model(original_image)
                if isinstance(output_mask, dict):
                    output_mask = output_mask['out']
                output_mask  = output_mask.squeeze(0).cpu()
                output_mask = convert_into_3d(output_mask) # decoding the output
                axs[2+a][i].imshow(output_mask)
            
    for a in range(5):
        for b in range(n_base + n_rand):
            axs[a][b].axis('off')
    

#     plt.subplots_adjust(left=0.1, bottom=0.1, right=0.7, top=0.9, wspace=0.3, hspace=0.3) 
    plt.savefig('image_comparison.png')
    plt.show() 

In [55]:
display_data(data_test)